In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from utils import Tokenizer
inputs = torch.load('../condqa_old/data/dev_inputs', map_location='cpu')
tokenizer = Tokenizer('../condqa_old/model')

In [65]:
from utils import TxtNode, get_level
import pickle
from functools import reduce
import numpy
import random
from copy import deepcopy
import os


def split_tokens(tokens):
    tokens = tokens.tolist()
    cutoffs = []
    current_cutoff = []
    for idx, i in enumerate(tokens):
        if i not in range(50265, 50272):
            current_cutoff.append((i, idx))
        else:
            cutoffs.append(current_cutoff)
            current_cutoff = [(i, idx)]
    cutoffs.append(current_cutoff)
    return cutoffs

def create_node_from_document(document_tokens):
    base_node = TxtNode([(0, -1)], tokenizer)
    nodes = [base_node]
    current_node = base_node
    for segment in document_tokens:
        while current_node.level >= get_level(segment):
            current_node = current_node.parent

        child = TxtNode(segment, tokenizer)
        nodes.append(child)
        current_node.children.append(child)
        child.parent = current_node
        current_node = child
    return base_node

def get_grouped_tokens(input_ids):
    splited_tokens = split_tokens(input_ids)
    start_tokens = splited_tokens[0]
    document_tokens = splited_tokens[1:-1]
    end_tokens = [i for i in splited_tokens[-1] if i[0] != 0]
    l_document_tokens = []
    while True:
        token = end_tokens.pop(0)
        if token[0] in list(range(50272, 50279)):
            l_document_tokens.append(token)
            break
        else:
            l_document_tokens.append(token)
    document_tokens.append(l_document_tokens)
    return start_tokens, document_tokens, end_tokens


def repeat_a_node(base_node):
    nodes = base_node.get_nodes_list()
    nodes = [node for node in nodes if node.parent != None]
    node = random.choice(nodes)
    node_new = node.copy()
    parent = node.parent
    node_new.parent = parent
    position = random.choice(list(range(len(parent.children) + 1)))
    parent.children = parent.children[:position] + [node_new] + parent.children[position:]
    return base_node

def remove_a_node(base_node):
    nodes = base_node.get_nodes_list()
    nodes = [node for node in nodes if node.parent != None]
    node = random.choice(nodes)
    parent = node.parent
    parent.children.remove(node)
    return base_node

def mask_a_node(base_node):
    nodes = base_node.get_nodes_list()
    node = random.choice(nodes)
    node.text = [(50264, j) for i, j in node.text]
    return base_node

def reorder_a_node(base_node):
    nodes = base_node.get_nodes_list()
    nodes = [node for node in nodes if len(node.children) > 1]
    node = random.choice(nodes)
    random.shuffle(node.children)
    return base_node



def recover_index_from_node(node):
    nodes = node.get_nodes_list()
    text = [i.text for i in nodes][1:]
    origin_HTMLelement_index = [i[0][1] for i in text]
    text_lengths = [len(i) for i in text]
    generated_HTMLelement_index = [0] + numpy.cumsum(text_lengths)[:-1].tolist()
    HTMLelement_index = list(zip(origin_HTMLelement_index, generated_HTMLelement_index))
    text = reduce(lambda x, y: x + y, text)
    return text, HTMLelement_index



In [101]:
from time import time

def contrastive_sampling(input_ids):
    start_tokens, document_tokens, end_tokens = get_grouped_tokens(input_ids)
    base_node = create_node_from_document(document_tokens)
    for i in range(len(base_node.get_nodes_list()) // 20):
        base_node = repeat_a_node(base_node)

    for i in range(len(base_node.get_nodes_list()) // 20):
        base_node = remove_a_node(base_node)

    for i in range(len(base_node.get_nodes_list()) // 5):
        base_node = reorder_a_node(base_node)

    for i in range(len(base_node.get_nodes_list()) // 10):
        base_node = mask_a_node(base_node)

    document_tokens, HTMLelement_index = recover_index_from_node(base_node)
    HTMLelement_index = torch.tensor(HTMLelement_index)
    document_tokens = start_tokens + document_tokens + end_tokens
    HTMLelement_index[:, 1] += len(start_tokens)

    return document_tokens, HTMLelement_index

def generate_contrastive_sample(input):
    sample = inputs[0]
    input_ids = sample[0]
    global_mask = sample[1]
    attention_mask = sample[2]
    mask_HTMLelements = sample[3]
    mask_label_HTMLelements = sample[4]
    mask_answer_span = sample[5]
    qa_id = sample[6]
    mask_label_condition = sample[7]

    new_input, contrastive_pairs = contrastive_sampling(input_ids)
    new_input_ids = torch.tensor([i[0] for i in new_input])
    arrangement_index = torch.tensor([i[1] for i in new_input])

    global_mask = global_mask[arrangement_index]
    attention_mask = attention_mask[arrangement_index]
    mask_HTMLelements = mask_HTMLelements[arrangement_index]
    mask_label_HTMLelements = mask_label_HTMLelements[arrangement_index]
    mask_answer_span = mask_answer_span[arrangement_index]
    mask_label_condition = mask_label_condition[arrangement_index]

    text_length = new_input_ids.shape[0]
    new_input_ids = torch.concat((new_input_ids, torch.ones(4000 - text_length, dtype = torch.long)))
    global_mask = torch.concat((global_mask, torch.zeros(4000 - text_length, dtype = torch.bool)))
    attention_mask = torch.concat((attention_mask, torch.zeros(4000 - text_length, dtype = torch.bool)))
    mask_HTMLelements = torch.concat((mask_HTMLelements, torch.zeros(4000 - text_length, dtype = torch.bool)))
    mask_label_HTMLelements = torch.concat((mask_label_HTMLelements, torch.zeros((4000 - text_length, 3), dtype = torch.long)))
    mask_answer_span = torch.concat((mask_answer_span, torch.zeros((4000 - text_length, 2), dtype = torch.long)))
    mask_label_condition = torch.concat((mask_label_condition, torch.zeros((4000 - text_length, 5, 2), dtype = torch.bool)))

    new_sample = [new_input_ids, global_mask, attention_mask, mask_HTMLelements, mask_label_HTMLelements, \
        mask_answer_span, qa_id, mask_label_condition]
    return new_sample, contrastive_pairs


In [102]:
A = inputs[0]
B, pair = generate_contrastive_sample(A)

In [74]:
max_HTML_num = 3
ans_indicator = torch.tensor(
    [[[0, 1], [0, 0], [0, 0]]]
)

cond_indicator = torch.tensor(
    [[[1, 0], [1, 1], [0, 0]]]
)
ans_indicator = ans_indicator.transpose(-2, -1)
cond_indicator = cond_indicator.transpose(-2, -1)
cond_indicator = cond_indicator.unsqueeze(2).repeat(1, 1, max_HTML_num, 1)# 1, 2, 3, 3
label_condition = torch.einsum('abcd,abc->acd',cond_indicator, ans_indicator)


In [75]:
label_condition

tensor([[[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]]])

In [72]:
label_condition

tensor([[[0, 1, 1],
         [1, 0, 1],
         [0, 0, 0]]])

In [57]:
ans_indicator = torch.tensor([[[1, 0, 0], [0, 0, 1]], [[0, 1, 0], [1, 0, 0]]])#1, 2, 3
cond_indicator = torch.tensor([[[0, 1, 1], [1, 1, 0]], [[1, 1, 1], [0, 0, 0]]])
cond_indicator = cond_indicator.unsqueeze(2).repeat(1, 1, 3, 1)# 1, 2, 3, 3
cond_indicator = torch.einsum('abcd,abc->acd',cond_indicator, ans_indicator)


In [71]:
original_mask = torch.ones(2, 3, 3, dtype=torch.long)
original_mask = original_mask.unsqueeze(2).repeat(1, 1, ans_indicator.shape[1], 1)
fixed_mask = torch.einsum('abcd,acd->adb', original_mask, ans_indicator)

In [72]:
fixed_mask

tensor([[[1, 1, 1],
         [0, 0, 0],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [0, 0, 0]]])

In [60]:
A = torch.tensor([[1, 1, 0]])
torch.einsum('ab,ac->abc',A, A)

tensor([[[1, 1, 0],
         [1, 1, 0],
         [0, 0, 0]]])

In [36]:
print([i['has_answer'] for i in examples[25]['document'] if i['has_answer'] != -1])


[3, 0, 1, 2, 0, 1, 2]


In [21]:
set(inputs[6][5].flatten().tolist())


{-1.0, 0.0, 1.0}

In [25]:
inputs[6][7][inputs[6][7]!=0]

tensor([1, 1, 1, 1, 1, 1])

In [31]:
for i in range(len(inputs)):
    for j in [1, 2, 3, 7]:
        inputs[i][j] = inputs[i][j].bool()

In [39]:
from utils import ReIndexer
indexer = ReIndexer()
hiddens = torch.arange(40).reshape(2, 10, 2)
mask = torch.rand(2, 10) > 0.8
mask = mask.float()
indexer.set_index(mask)
new_hiddens, new_mask = indexer.re_index(hiddens)


In [45]:
recovered_hiddens = 

tensor([[4, 0, 1, 2, 3, 5, 6, 7, 8, 9],
        [1, 5, 0, 2, 3, 4, 6, 7, 8, 9]])

In [49]:
recovered_hiddens = torch.empty_like(first_indexer.index.unsqueeze(-1).repeat(1, 1, 2)).scatter_(dim=1, index=first_indexer.index.unsqueeze(-1).repeat(1, 1, 2), src=new_hiddens)

In [30]:
from utils import convert_examples_to_inputs, Tokenizer
tokenizer = Tokenizer('../condqa_old/model')
inputs = convert_examples_to_inputs(examples, tokenizer)

converting examples to inputs..: 100%|██████████| 352/352 [00:01<00:00, 182.42it/s]


In [32]:
torch.save(inputs, '../condqa_old/data/dev_inputs')

In [33]:
for i in inputs[0]:
    print(i.dtype, i.shape)

torch.int64 torch.Size([4000])
torch.bool torch.Size([4000])
torch.bool torch.Size([4000])
torch.bool torch.Size([4000])
torch.float32 torch.Size([4000, 3])
torch.float32 torch.Size([4000, 2])
torch.int64 torch.Size([])
torch.bool torch.Size([4000, 5, 2])


In [4]:
inputs = torch.load('../condqa_old/data/dev_inputs')

In [55]:
inputs[38][-2]

tensor(29)

In [59]:
inputs[39][-1].nonzero()


tensor([[ 969,    0,    1],
        [1142,    0,    0]])